In [21]:
import sys  
sys.path.insert(0, './../')
import numpy as np
import bbdc2021 as bbdc
import pandas as pd
from importlib import reload
reload(bbdc)
import tensorflow as tf
from keras import layers
import keras
from keras import backend as K
import matplotlib.pyplot as plt
from matplotlib import colors 
import evaluation.evaluate as evaluate

In [2]:
np.random.seed(1)
tf.random.set_seed(1)

In [23]:
labelDf = pd.read_csv("./../google_audioset/googleData/fft/balanced_train_segments.csv")
labelDf['event_label'] = labelDf['event_label'].apply(ast.literal_eval)
# Zuerst zu numerischen Werten per Dictionary konvertieren,
# um zu vermeiden bei fehlenden Labels falsche Decodings zu erzeugen
#labelDf = labelDf.replace({"event_label": LABEL_DICT})

In [22]:
#X_train, Y_train, trainFileList = bbdc.load_audioset("unbalanced_train_segments.csv", "unbalanced_fft_mel_l04_o026_f32/", pathToDataDir="./../google_audioset/googleData/fft/")
#X_validation, Y_validation, validationFileList = bbdc.load_audioset("balanced_train_segments.csv", "balanced_fft_mel_l04_o026_f32/", pathToDataDir="./../google_audioset/googleData/fft/")
X_test, Y_test, testFileList = bbdc.load_audioset("eval_segments.csv", "eval_fft_mel_l04_o026_f32/", pathToDataDir="./../google_audioset/googleData/fft/")

631it [00:13, 47.00it/s]


In [24]:
X_test[1].shape

(768, 32)

In [9]:
channels = [40, 50]
numClasses=13

input_layer = layers.Input(shape=(None, 32))
x = input_layer
x = layers.Conv1D(channels[0], kernel_size=3, padding='valid', activation='relu')(x)
x = layers.Conv1D(channels[0], kernel_size=3, padding='valid', activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling1D()(x)

x = layers.Conv1D(channels[1], kernel_size=3, padding='valid', activation='relu')(x)
x = layers.Conv1D(channels[1], kernel_size=3, padding='valid', activation='relu')(x)
x = layers.Conv1D(channels[1], kernel_size=3, padding='valid', activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling1D()(x)

x = layers.Conv1D(channels[0], kernel_size=3, padding='valid', activation='relu')(x)
x = layers.Conv1D(channels[0], kernel_size=3, padding='valid', activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Conv1D(numClasses, kernel_size=1, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)

model = keras.models.Model(inputs=input_layer, outputs=x)
model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, None, 32)]        0         
_________________________________________________________________
conv1d_52 (Conv1D)           (None, None, 40)          3880      
_________________________________________________________________
conv1d_53 (Conv1D)           (None, None, 40)          4840      
_________________________________________________________________
batch_normalization_21 (Batc (None, None, 40)          160       
_________________________________________________________________
max_pooling1d_16 (MaxPooling (None, None, 40)          0         
_________________________________________________________________
conv1d_54 (Conv1D)           (None, None, 50)          6050      
_________________________________________________________________
conv1d_55 (Conv1D)           (None, None, 50)          7550

In [26]:
opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['mae', 'accuracy'])
history = model.fit(X_test, Y_test, batch_size=1, epochs=10, shuffle=True)

ValueError: Data cardinality is ambiguous:
  x sizes: 353
  y sizes: 13
Make sure all arrays contain the same number of samples.

In [14]:
X_test.dtype

dtype('O')